# Imports

In [34]:
import numpy as np
import _pickle as pickle
import gym
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.initializers as initializers

# Model

In [102]:
initializer = initializers.GlorotNormal

x_in = layers.Input(shape = (6400,1))
x = layers.Dense(200, kernel_initializer= initializer, activation="relu")(x_in)
x_out = layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(x_in, x_out)

model.compile(optimizer = "adam", loss = "mse", metrics = ["acc"])

In [49]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [71]:
env.unwrapped.get_action_meanings()

# NOOP is the same as FIRE (standing still)
# LEFT is the same as LEFTFIRE (down)
# RIGHT is the same as RIGHTFIRE (up)

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [61]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 32, 1)]           0         
_________________________________________________________________
dense_24 (Dense)             (None, 32, 200)           400       
_________________________________________________________________
dense_25 (Dense)             (None, 32, 1)             201       
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


# Preprocessing

In [4]:
def prepro(input_frame):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    input_frame = input_frame[34:194] # crop
    input_frame = input_frame[::2,::2,0] # downsample by factor of 2 (halves the resolution of the image)
    #This takes every other pixel in the image
    input_frame[input_frame == 144] = 0 # erase background (background type 1)
    input_frame[input_frame == 109] = 0 # erase background (background type 2)
    input_frame[input_frame != 0] = 1 # everything else (paddles, ball) just set to 1
    return input_frame.astype(np.float).ravel()

# Hyperparamters

In [5]:
render = False
prev_frame = None
game_dimensions = 80*80
gamma = 0.99

In [82]:
print(change_in_frame)

[0. 0. 0. ... 0. 0. 0.]


In [103]:
x1 = np.asarray(total_change_in_frames)
y1 = np.asarray(total_rewards)

model.fit(x = x1, y = y1)

# Must all be numpy arrays

32/32 [==============================] - 11s 318ms/step - loss: 0.2451 - acc: 0.9799


TypeError: predict() missing 1 required positional argument: 'x'

In [75]:
app = tf.convert_to_tensor(change_in_frame)
app.shape

TensorShape([6400])

# Main Code

In [95]:
env = gym.make("Pong-v0")
observation = env.reset()
reward_sum = 0
eps = 1
episode_number = 0
running_reward = None
losses = []
total_change_in_frames, total_rewards = [], []

while True:
    if render:
        env.render()
    
    curr_frame = prepro(observation)
    change_in_frame = curr_frame - prev_frame if prev_frame is not None else np.zeros(game_dimensions)
    prev_frame = curr_frame
    total_change_in_frames.append(change_in_frame)
    
    """if np.random.random() < eps:
        action = env.action_space.sample()
    else:
        _, action = Q_net.maximum(torch.as_tensor(change_in_frame).float())
        action = action.numpy()"""
    
    
    
    observation, reward, done, _ = env.step(action) 
    total_rewards.append([reward])
    reward_sum += reward
    eps -= 0.001
    
    if done:
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print ('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_frame = None
        episode_number += 1
    
    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000


In [73]:
env.render()

KeyboardInterrupt: 